In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [11]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    image_position = ds.get("PatientPosition", "Not Found")
    image_type = ds.get("ImageType", "Not Found")
    body_part_examined = ds.get("BodyPartExamined", "Not Found")
    print('image_position',image_position)
    print('image_type',image_type)
    print('body_part_examined',body_part_examined)
    #for elem in ds:
    #    if((elem.name == ''))
    #    print(f"{elem.tag} - {elem.name}: {elem.value}")
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def resize_image(img, target_size):
    # Resize the image to the target size using NumPy (same as before)
    original_height, original_width = img.shape[:2]
    target_height, target_width = target_size

    resized_img = np.zeros((target_height, target_width), dtype=img.dtype)

    height_scale = original_height / target_height
    width_scale = original_width / target_width

    for i in range(target_height):
        for j in range(target_width):
            orig_x = int(j * width_scale)
            orig_y = int(i * height_scale)
            resized_img[i, j] = img[orig_y, orig_x]

    return resized_img

def preprocess_image(img, img_size=(224, 224)):
    # Resize the image to the desired size (224x224)
    resized_image = resize_image(img, img_size)

    # Normalize the image to [0, 1]
    normalized_image = resized_image.astype(np.float32) / 255.0


    normalized_image = np.stack((normalized_image,) * 3, axis=-1)  # Shape becomes (height, width, 3)

    # Add batch dimension (1, height, width, channels)
    normalized_image_array = np.expand_dims(normalized_image, axis=0)  # Add batch dimension

    return normalized_image_array

# This function loads in our trained model w/ weights and compiles it 
def load_my_model(model_path, weight_path):
    # todo
    model = load_model(model_path)
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    prediction = model.predict(img)
    print('prediction', prediction)
    return (prediction >= thresh)

In [12]:
check_dicom('test1.dcm')

Load file test1.dcm ...
image_position PA
image_type Not Found
body_part_examined CHEST


array([[199, 175, 152, ..., 252, 251, 251],
       [150, 128, 114, ..., 243, 250, 252],
       [122, 110, 102, ..., 193, 222, 239],
       ...,
       [ 44,  47,  51, ...,  16,  16,  13],
       [ 44,  48,  52, ...,  16,  16,  14],
       [ 45,  48,  53, ...,  17,  17,  15]], dtype=uint8)

In [13]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model_vgg16_model.h5'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(224,224)#(1,224,224,3) # This might be different if you did not use vgg16
#img_mean = # loads the mean image value they used during training preprocessing
#img_std = # loads the std dev image value they used during training preprocessing

my_model = load_my_model(model_path, weight_path)
thresh = 0.7964839

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
image_position PA
image_type Not Found
body_part_examined CHEST
prediction [[0.4091284]]
[[False]]
Load file test2.dcm ...
image_position AP
image_type Not Found
body_part_examined CHEST
prediction [[0.34331533]]
[[False]]
Load file test3.dcm ...
image_position AP
image_type Not Found
body_part_examined CHEST
prediction [[0.7490935]]
[[False]]
Load file test4.dcm ...
image_position PA
image_type Not Found
body_part_examined RIBCAGE
prediction [[0.4091284]]
[[False]]
Load file test5.dcm ...
image_position PA
image_type Not Found
body_part_examined CHEST
prediction [[0.4091284]]
[[False]]
Load file test6.dcm ...
image_position XX
image_type Not Found
body_part_examined CHEST
prediction [[0.4091284]]
[[False]]
